## We Need to Talk + MIT Code for Good '22
This notebook reads from this [data spreadsheet](https://docs.google.com/spreadsheets/d/1_OsK5jXUoQP0JRrfKKwzCxPS936-Qp3fE6RgNR_a82I/edit#gid=2114958450) that our CFG team has gathered, and utilizes a simple model to calculate period poverty scores across 81 different provinces in Turkey. In the future, as more data is obtained, the model and spreadsheet can be modified to accomodate for these changes.

The following links are helpful to get kickstarted with the Google Sheets API:
- https://developers.google.com/sheets/api/quickstart/python
- https://blog.coupler.io/python-to-google-sheets/

This notebook requires:
- pandas
- google-auth 2.3.3
- google-api-python-client 2.35.0
- google-api-core 2.4.0
- google-auth-oauthlib 0.4.6

In [4]:
from googleapiclient.discovery import build
from google.oauth2 import service_account
from googleapiclient.errors import HttpError
import pandas as pd
import json
from collections import defaultdict

In [8]:
# Scope to allow read/write to the service account's files
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = "we-need-to-talk-338617-1bfc415b1e1b.json"

CREDENTIALS = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
SPREADSHEET_ID = "1_OsK5jXUoQP0JRrfKKwzCxPS936-Qp3fE6RgNR_a82I"

In [9]:
# Try an example first

SHEET_RANGE = "Data!A1:M82"
try:
    service = build('sheets', 'v4', credentials=CREDENTIALS)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=SHEET_RANGE).execute()
    values = result.get('values', [])

    if not values:
        raise Exception('No data found.')

except HttpError as err:
    print(err)

In [10]:
df = pd.DataFrame(values[1:], columns=values[0])
df

,Province Name,Region,Phone Prefix,Population (2019-2020 Estimate),Number of Menstruators (Estimate),Number of Refugee Menstruators (Estimate),Period Poverty Score
0,Adana,Mediterranean,322,"2,237,940","680,581","72,648",4.72
1,Adıyaman,Southeastern Anatolia,416,"626,465","190,515","6,496",5.47
2,Afyonkarahisar,Aegean,272,"729,483","221,843","3,532",2.90
3,Ağrı,Eastern Anatolia,472,"536,199","163,064",357,5.85
4,Aksaray,Central Anatolia,382,"416,567","126,682","1,121",3.96
...,...,...,...,...,...,...,...
76,Uşak,Aegean,276,"370,509","112,676",889,3.22
77,Van,Eastern Anatolia,432,"1,136,757","345,700",620,6.97
78,Yalova,Marmara,226,"270,976","82,407","1,116",1.92
79,Yozgat,Central Anatolia,354,"421,200","128,091","1,499",3.45


In [14]:
MONTHLY_MENSTRUAL_COSTS = 200  # Assuming the purchase of pads, units in Turkish Liras (TRY)
AVG_NUM_FEMALES_PER_HOUSEHOLD = 2  # TODO: Remove this + alter model code below if using per person vs per household

def setup_sheets_api_client(creds):
    """
    Returns the sheets api client. 
    The client object can be called as follows to read from a spreadsheet:
    
    client.values().get(spreadsheetId=xyzid, range=xyzrange).execute()
    
    Raises HttpError if the connection fails.
    """
    service = build('sheets', 'v4', credentials=creds)
    return service.spreadsheets()

def add_sheet_data_to_dict(sheets_api_client, sheet_range, data_dict):
    """
    Uses sheets api client to read a specified range from the data spreadsheet
    (Id can be found in the URL: https://docs.google.com/spreadsheets/d/<ID HERE>/edit#gid=blah).
    Adds the data spanning the range to data_dict in the format: 
    {
        Adana: {
            "Region": "Mediterranean", "Population": 200, ...
        },
        Istanbul: {
            "Region": "Marmara", ...
        }, ...
    }
    
    Raises Exception if no data is found, or HttpError if the connection fails.
    """
    result = sheets_api_client.values().get(spreadsheetId=SPREADSHEET_ID, range=sheet_range).execute()
    values = result.get('values', [])

    if not values:
        raise Exception('No data found.')

    df = pd.DataFrame(values[1:], columns=values[0])
    
    for i in range(len(df)):
        province = df.iloc[i]["Province Name"]
        for col in df.columns:
            data_dict[province][col] = df.iloc[i][col]
            
def calculate_period_poverty_per_province(data_dict, c1=2.5, c2=1, c3=4, c4=2.5):
    """
    Calculates and stores period poverty score by province in data_dict.
    Tentative formula:
    
    c1 * di + c2 * sr + c3 * pu + c4 * hc
    where (all values are calculated estimates or data taken from reputable organizations):
        - c1, c2, c3, and c4 are tunable coefficients
        - di := distress index (HDI) for females
        - sr := syrian refugee percentage
        - pu := period unaffordability index (calculated via monthly menstrual expenditures vs monthly income)
        - hc := lack of healthcare index (calculated via availability of hospitals and staff)
        
    Default values for c1, c2, c3, c4 chosen arbitrarily. These can and should be adjusted.
    """
    for province in data_dict:
        di = float(data_dict[province]["Distress Index Females"])
        sr = float(data_dict[province]["Syrian Refugees (%)"].replace("%", ""))  # Remove percent sign
        pu = float(data_dict[province]["Period Unaffordability Index"])
        hc = float(data_dict[province]['Lack of Healthcare Index'])
        
        data_dict[province]["Period Poverty Score"] = c1 * di + c2 * sr + c3 * pu + c4 * hc

#### Notes:
- The Well-Being Index comes from the [Human Development Indices project](https://globaldatalab.org/shdi/2019/gender-development/TUR/?levels=1%2B4&interpolation=1&extrapolation=0&nearest_real=0) with its technical details explained [here](http://hdr.undp.org/sites/default/files/hdr2020_technical_notes.pdf).
- Mean income data taken from [here](https://data.tuik.gov.tr/Bulten/Index?p=Income-and-Living-Conditions-Survey-Regional-Results-2020-37405).
- See the Sources tab in our master spreadsheet for more details on how we obtained the data from international organizations' online public data.

In [32]:
if __name__ == "__main__":
    client = setup_sheets_api_client(CREDENTIALS)
    data_dict = defaultdict(dict)
    
    # Add more tabs and columns to this list to change the information processed by the model
    # Some sheets have specific row ranges since there's a summary set of cells at the bottom that we filter out
    relevant_sheet_tab_ranges = [
        "Data!A1:F82",
        "Distress Index!A:G",
        "Syrian Refugees Data!A1:E82",
        "Income/Expenditure!A1:K82",
        "Lack of Health Care!A1:J82"
    ]
    
    # Read in all relevant data into data_dict
    for sheet_range in relevant_sheet_tab_ranges:
        add_sheet_data_to_dict(client, sheet_range, data_dict)

    # Use our model to calculate the period poverty score per region
    calculate_period_poverty_per_province(data_dict)
    
    # Save results into a json file
    with open("provinces_data.json", "w") as f:
        json.dump(data_dict, f)